# Experiment 1
### In this experiment we compare the predictive capabilities of three meta models across four types of dataset. 
We compare the new amortised global inducing point BNN with a vanilla neural process and an amortised MFVI BNN as two baseline models. The test datasets are:
- Noisy sample from squared exponential covarince GP prior
- Noisy sample from Laplacian covariance GP prior
- Noisy sample from periodic covariance GP prior
- Noisy cubic dataset with central gap

For each dataset, a non meta model representing the 'ground truth' is used to compare predictions against. For the GP-generated datasets, the posterior from a hyperparameter-optimised GP with the corresponding covariance function is used, and for the cubic dataset, the vanilla global inducing point BNN is used.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import matplotlib.pyplot as plt
import gpytorch
import tqdm.auto as tqdm
import sys

torch.set_default_dtype(torch.float64)

sys.path.append("../")
from bnn_amort_inf.models.bnn import gibnn
from bnn_amort_inf.models import gp, np
from bnn_amort_inf import utils

### Generate metadatasets

In [ ]:
num_datasets = 1000
meta_datasets = {}
kernels = ["se", "per", "lap"]

for kernel in kernels:
    meta_datasets[kernel] = utils.dataset_utils.MetaDataset(
        [
            utils.gp_datasets.gp_dataset_generator(min_n=30, max_n=100, kernel=kernel)
            for _ in range(num_datasets)
        ]
    )
meta_datasets["mixed"] = [
    utils.gp_datasets.gp_dataset_generator(min_n=30, max_n=100, kernel=kernels[i % 3])
    for i in range(num_datasets)
]

### Generate test datasets

In [ ]:
test_datasets = {}

seed = 42

# TODO: implement seed parameter for gp dataset creation
for kernel in kernels:
    test_datasets[kernel] = torch.utils.data.TensorDataset(
        utils.gp_datasets.gp_dataset_generator(min_n=15, max_n=20, kernel=kernel)
    )

# TODO: move cubic dataset creation to dataset_utils